In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score 
import matplotlib.pyplot as plt
import re

# CSV 파일에서 데이터 로드
file_path = '20240703_correlation.csv'
df = pd.read_csv(file_path)

# 열 목록
columns = df.columns

# R^2 값을 저장할 행렬 초기화
r2_matrix = pd.DataFrame(index=columns, columns=columns)

# 각 열을 종속 변수로 지정
for dependent_var in columns:
    independent_vars = df.columns.difference([dependent_var])
    
    for var in independent_vars:
        # 결측치 제거
        valid_data = df[[var, dependent_var]].dropna()
        
        X = valid_data[[var]]
        y = valid_data[dependent_var]
        
        if len(valid_data) > 0:  # 유효한 데이터가 있는지 확인
            model = LinearRegression()
            model.fit(X, y)
            y_pred = model.predict(X)
            
            r2 = r2_score(y, y_pred)
            r2_matrix.loc[var, dependent_var] = r2
        else:
            r2_matrix.loc[var, dependent_var] = np.nan  # 유효한 데이터가 없으면 NaN으로 설정

# r2_matrix를 CSV 파일로 저장
output_file_path = '20240703_r2_matrix.csv'  # 저장할 파일 경로
r2_matrix.to_csv(output_file_path)

# R^2 값이 0.9 이상인 모델 찾기
high_r2_models = r2_matrix[r2_matrix >= 0.90].stack().reset_index()
high_r2_models.columns = ['Independent_Var', 'Dependent_Var', 'R2']

# 첫 번째 모델에 대해 CSV 파일과 그래프 저장
if not high_r2_models.empty:
    row = high_r2_models.iloc[0]  # 첫 번째 모델 선택
    independent_var = row['Independent_Var']
    dependent_var = row['Dependent_Var']
    
    # 결측치 제거
    valid_data = df[[independent_var, dependent_var]].dropna()
    
    X = valid_data[[independent_var]]
    y = valid_data[dependent_var]
    
    # 모델 학습 및 예측
    model = LinearRegression()
    model.fit(X, y)
    y_pred = model.predict(X)
    
    # 실제 값과 예측치 저장
    result_df = pd.DataFrame({
        'Actual': y,
        'Predicted': y_pred
    })
    
    # 파일 이름에서 유효하지 않은 문자 제거
    def sanitize_filename(filename):
        return re.sub(r'[\\/*?:"<>|]', "_", filename)

    csv_filename = sanitize_filename(f'{independent_var}_{dependent_var}.csv')
    result_df.to_csv(csv_filename, index=False)
    

    # 그래프 생성
    plt.plot([y.min(), y.max()], [y.min(), y.max()], 'k-', lw=2)
    plt.scatter(y, y_pred, color = 'red', alpha=0.5)
    plt.xlabel(f'Actual values of {dependent_var}')
    plt.ylabel('Predicted values of {dependent_var}')
    plt.title(f'Model {independent_var}_{dependent_var}')

    # 그래프 이미지 파일로 저장
    img_filename = sanitize_filename(f'{independent_var}_{dependent_var}.png')
    plt.savefig(img_filename)
    plt.close()